In [1]:
from odebench import ConfigParser
from odebench.data_sampler import ZeroDSampler, OneDSampler
from odebench.dataset_tools import DatasetMerger, DataAugmenter
from odebench.dataset_tools.container import Container
from odebench.nn_framework.model import MLPModel
from odebench.utils import visualiser

import torch
import torch.nn as nn
import numpy as np

TypeError: unsupported operand type(s) for |: 'type' and 'NoneType'

data sample

In [2]:
sampler = OneDSampler()
sampler.sample()
data = sampler.get_data()
sampler.save()
sampler.clean()

Loaded config for fuel: H2

 Starting sampling pipeline...
Calculating steady flame properties with Cantera...
Solving premixed flame...
Laminar Flame Speed      :   2.3489328863 m/s
Laminar Flame Thickness  :   0.0003694362 m
Flame Thickness: 3.69e-04
Flame Speed: 2.35e+00
Domain Width: 1.85e-03
Domain Length: 1.85e-02
Half Domain Length: 9.24e-03
Target Time Step: 1.00e-06
Chemical Time Scale: 1.57e-04
Sample Time Steps: 1.00e+02
Estimated Time Step: 1.57e-05
Estimated Sim Time: 1.59e-03
Estimated Write Time Step: 1.57e-05
✅ Updated CanteraMechanismFile to /root/ODEBench/mechanisms/Burke2012_s9r23.yaml
⚙️ Running Allrun script...
Error running Allrun!
Return code: 127
STDOUT:

STDERR:
./Allrun: 5: .: Can't open /bin/tools/RunFunctions



CalledProcessError: Command '['./Allrun']' returned non-zero exit status 127.

In [ ]:
config_0d = ConfigParser.load_config("config/0d_config.yaml")
sampler_0d = ZeroDSampler(config_0d)
sampler_0d.sample()
data_0d = sampler_0d.data()

config_1d = ConfigParser.load_config("config/1d_config.yaml")
sampler_1d = OneDSampler(config_1d)
sampler_1d.sample()
data_1d = sampler_1d.data

data preprocess

In [ ]:
merged_data = DatasetMerger.merge([data_0d, data_1d])
augmented_data = DataAugmenter.add_multi_scale_perturbation(merged_data)
# other data processing operations can be added here

data train

In [ ]:
featureSize = 10
labelSize = 1 # Constrained to one when using default Container
totalRecords = 1000

epochs: int = 10
batch_size: int = 64
lr: float = 1e-3

# Define data and feed into Container
data: np.ndarray = np.random.random((totalRecords, featureSize+labelSize))
dataContainer: Container = Container(data)

# Shuffle and split data
dataContainer.shuffle()
container_train, container_vali = dataContainer.train_test_split(0.8)

# Create and train model
model: MLPModel = MLPModel([featureSize, 64, 64, labelSize])
model.set_optim(torch.optim.Adam(model.parameters(), lr=lr))
train_loss, val_loss = model.fit(container_train, container_vali, epochs, batch_size, verbose=True)

# TODO: val_loss is not list[float] but list[list[float]] as there may be multiple metrics.
visualiser.plot_loss_curve(train_loss, val_loss)